In [16]:
import cv2
import torch
import json
from ultralytics import YOLO
from datetime import timedelta
import argparse
import time
import os


video_path= 'C:/Users/alex_dextop/PycharmProjects/Yolo_2/rem.mp4'      # путь к видеофайлу
MODEL_PATH = 'yolov8n.pt'          # путь к твоей модели
OUTPUT_JSON = 'violations_report.json'
VIOLATION_CLASSES = {
    0: 'No_Helmet',
    1: 'No_Vest',
    2: 'Unsafe_Position',
    # Добавь свои классы если нужно
}
model = YOLO(MODEL_PATH)
cap = cv2.VideoCapture(video_path)
fps = cap.get(cv2.CAP_PROP_FPS)
interval = 5  # секунд

results_by_interval = []
frame_num = 0
current_interval_start = 0
interval_violations = []



def save_interval_result(start_frame, detected_violations):
    if not detected_violations:
        return None
    # Сгруппируем классы и объекты
    grouped = {}
    for item in detected_violations:
        key = (item['violation_class'], tuple(item['detected_objects']))
        grouped.setdefault(key, []).append(item)

    result = []
    for (vcls, dobjs), group in grouped.items():
        result.append({
            "violation_class": vcls,
            "number": '',  # Место для распознавания номеров
            "frame": group[0]['frame'],
            "time": f"{group[0]['time']}",
            "detected_objects": list(dobjs),
            "violation_type": "Safety",
            "count": len(group)
        })
    return result







class YOLOv8PersonDetector:
    def __init__(self, model_path='yolov8n.pt', conf_threshold=0.5):
        """
        Инициализация детектора
        
        Args:
            model_path (str): Путь к модели YOLOv8
            conf_threshold (float): Порог уверенности для детекции
        """
        self.conf_threshold = conf_threshold
        
        # Загрузка модели
        print("Загрузка модели YOLOv8...")
        self.model = YOLO(model_path)
        print("Модель загружена успешно!")
        
        # Проверяем наличие GPU
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        print(f"Используется устройство: {self.device}")
        
    def process_video(self, video_path, output_path=None, show_video=True):
        """
        Обработка видео и обнаружение людей
        
        Args:
            video_path (str): Путь к входному видео
            output_path (str): Путь для сохранения результата (опционально)
            show_video (bool): Показывать ли видео в реальном времени
        """
        # Открываем видео
        # cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            print(f"Ошибка: Не удалось открыть видео {video_path}")
            return
        
        # Получаем параметры видео
        fps = cap.get(cv2.CAP_PROP_FPS)
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        
        print(f"Параметры видео: {width}x{height}, FPS: {fps:.2f}, Кадров: {total_frames}")
        
        # Настройка видео writer для сохранения результата
        if output_path:
            fourcc = cv2.VideoWriter_fourcc(*'mp4v')
            out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
        
        frame_count = 0
        start_time = time.time()
        
        print("Начинаем обработку видео...")
        
        while True:
            ret, frame = cap.read()
            cv2.line(frame, (750, 715), (625, 170), (0, 0, 255), 2, 2)

            if not ret:
                break

            frame_num += 1
            time_in_sec = frame_num / fps 
            if time_in_sec >= current_interval_start + interval:
                v = save_interval_result(current_interval_start, interval_violations)
                if v:
                    results_by_interval.extend(v)
                interval_violations = []
                current_interval_start += interval
            results = model(frame)[0]

            for det in results.boxes.data.tolist():
                    x1, y1, x2, y2, conf, cls_id = det
                    cls_id = int(cls_id)
                    violation_class = VIOLATION_CLASSES.get(cls_id, f'class_{cls_id}')
                    objects = ['person'] if violation_class.startswith('No_') or violation_class == 'Unsafe_Position' else []
                    time_str = str(timedelta(seconds=int(time_in_sec)))
                    violation = {
                        "violation_class": violation_class,
                        "number": "",  # номер можно дописать если появится модуль чтения номеров
                        "frame": frame_num,
                        "time": time_str,
                        "detected_objects": objects,
                        "violation_type": "Safety"
                    }
                    interval_violations.append(violation)
            v = save_interval_result(current_interval_start, interval_violations)
            if v:
                results_by_interval.extend(v)



            # Детекция людей на кадре
            results = self.model(frame, conf=self.conf_threshold, classes=[0])  # class 0 = person
            
            # Отрисовка результатов
            annotated_frame = self.draw_detections(frame, results)
            
            # Отображение информации
            annotated_frame = self.display_info(annotated_frame, frame_count, fps)
            
            # Сохранение результата
            if output_path:
                out.write(annotated_frame)
            
            # Показ видео
            if show_video:
                cv2.imshow('YOLOv8 Person Detection', annotated_frame)
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break
            
            frame_count += 1
            
            # Вывод прогресса каждые 100 кадров
            if frame_count % 100 == 0:
                elapsed_time = time.time() - start_time
                print(f"Обработано кадров: {frame_count}/{total_frames} "f"({frame_count/elapsed_time:.2f} FPS)")
        
        # Освобождение ресурсов
        cap.release()
        if output_path:
            out.release()
        cv2.destroyAllWindows()
        
        total_time = time.time() - start_time
        print(f"Обработка завершена!")
        print(f"Всего кадров: {frame_count}")
        print(f"Общее время: {total_time:.2f} сек")
        print(f"Средняя скорость: {frame_count/total_time:.2f} FPS")
    
    def draw_detections(self, frame, results):
        """
        Отрисовка bounding boxes и меток на кадре
        """
        for result in results:
            boxes = result.boxes
            if boxes is not None:
                for box in boxes:
                    # Координаты bounding box
                    x1, y1, x2, y2 = map(int, box.xyxy[0])
                    conf = box.conf[0].item()
                    class_id = int(box.cls[0])
                    
                    # Рисуем bounding box
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                    
                    # Добавляем метку с уверенностью
                    label = f"Person: {conf:.2f}"
                    label_size = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2)[0]
                    
                    # Фон для текста
                    cv2.rectangle(frame, (x1, y1 - label_size[1] - 10), (x1 + label_size[0], y1), (0, 255, 0), -1)
                    
                    # Текст
                    cv2.putText(frame, label, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 2)
        
        return frame
    
    def display_info(self, frame, frame_count, fps):
        """
        Отображение информации на кадре
        """
        info_text = f"Frame: {frame_count} | FPS: {fps:.1f}"
        cv2.putText(frame, info_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
        return frame

def main():
    parser = argparse.ArgumentParser(description='YOLOv8 Person Detection on Video')
    parser.add_argument('--video', type=str, required=True, help='Path to input video')
    parser.add_argument('--output', type=str, default='output.mp4', help='Path to output video')
    parser.add_argument('--model', type=str, default='yolov8m.pt', help='Path to YOLOv8 model')
    parser.add_argument('--conf', type=float, default=0.5, help='Confidence threshold')
    parser.add_argument('--no-show', action='store_true', help='Do not show video during processing')
    
    args = parser.parse_args()
    
    # Проверка существования видеофайла
    if not os.path.exists(args.video):
        print(f"Ошибка: Видеофайл {args.video} не найден!")
        return
    
    # Создание детектора
    detector = YOLOv8PersonDetector(model_path=args.model, conf_threshold=args.conf)
    
    # Обработка видео
    detector.process_video(
        video_path=args.video,
        output_path=args.output,
        show_video=not args.no_show
    )

# Простой вариант использования без аргументов командной строки
def simple_example():
    """
    Простой пример использования без аргументов командной строки
    """
    # video_path = "C:/Users/alex_dextop/PycharmProjects/Yolo_2/remont_1.mp4"  # Укажите путь к вашему видео
    output_path = "detected_people.mp4"
    
    # Проверка существования файла
    if not os.path.exists(video_path):
        print(f"Видеофайл {video_path} не найден!")
        print("Пожалуйста, укажите правильный путь к видеофайлу")
        return
    
    # Создание детектора
    detector = YOLOv8PersonDetector(model_path='yolov8m.pt', conf_threshold=0.5)
    
    # Обработка видео
    detector.process_video(
        video_path=video_path,
        output_path=output_path,
        show_video=True
    )

with open(OUTPUT_JSON, "w", encoding="utf-8") as f:
    json.dump(results_by_interval, f, ensure_ascii=False, indent=4)

print(f"Сохранено {len(results_by_interval)} записей о нарушениях в {OUTPUT_JSON}")

if __name__ == "__main__":
    # Использование с аргументами командной строки
    # main()
    
    # Или простой вариант (раскомментируйте нужный)
    simple_example()

Сохранено 0 записей о нарушениях в violations_report.json
Загрузка модели YOLOv8...
Модель загружена успешно!
Используется устройство: cpu
Параметры видео: 1280x720, FPS: 29.97, Кадров: 315
Начинаем обработку видео...


UnboundLocalError: cannot access local variable 'frame_num' where it is not associated with a value

In [23]:
import cv2
import torch
from ultralytics import YOLO
import argparse
import time
import os
import sort

class YOLOv8PersonDetector:
    def __init__(self, model_path='yolov8n.pt', conf_threshold=0.5):
        """
        Инициализация детектора
        
        Args:
            model_path (str): Путь к модели YOLOv8
            conf_threshold (float): Порог уверенности для детекции
        """
        self.conf_threshold = conf_threshold
        
        # Загрузка модели
        print("Загрузка модели YOLOv8...")
        self.model = YOLO(model_path)
        print("Модель загружена успешно!")
        
        # Проверяем наличие GPU
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        print(f"Используется устройство: {self.device}")
        # self.tracker = sort.Sort()
        
    def process_video(self, video_path, output_path=None, show_video=True):
        """
        Обработка видео и обнаружение людей
        
        Args:
            video_path (str): Путь к входному видео
            output_path (str): Путь для сохранения результата (опционально)
            show_video (bool): Показывать ли видео в реальном времени
        """
        # Открываем видео
        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            print(f"Ошибка: Не удалось открыть видео {video_path}")
            return
        
        # Получаем параметры видео
        fps = cap.get(cv2.CAP_PROP_FPS)
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        
        print(f"Параметры видео: {width}x{height}, FPS: {fps:.2f}, Кадров: {total_frames}")
        
        # Настройка видео writer для сохранения результата
        if output_path:
            fourcc = cv2.VideoWriter_fourcc(*'mp4v')
            out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
        
        frame_count = 0
        start_time = time.time()
        
        print("Начинаем обработку видео...")
        
        while True:
            ret, frame = cap.read()
            cv2.line(frame, (750, 715), (625, 170), (0, 0, 255), 2, 2)
            if not ret:
                break
            
            # Детекция людей на кадре
            results = self.model(frame, conf=self.conf_threshold, classes=[0])  # class 0 = person
            
            # Отрисовка результатов
            annotated_frame = self.draw_detections(frame, results)
            
            # Отображение информации
            annotated_frame = self.display_info(annotated_frame, frame_count, fps)

            # Сохранение результата
            if output_path:
                out.write(annotated_frame)
            
            # Показ видео
            if show_video:
                cv2.imshow('YOLOv8 Person Detection', annotated_frame)
                if cv2.waitKey(1) & 0xFF == ord('q'):
                    break
            
            frame_count += 1
            
            # Вывод прогресса каждые 10 кадров
            if frame_count % 1000 == 0:
                elapsed_time = time.time() - start_time
                print(f"Обработано кадров: {frame_count}/{total_frames} "  f"({frame_count/elapsed_time:.2f} FPS)")
        
        # Освобождение ресурсов
        cap.release()
        if output_path:
            out.release()
        cv2.destroyAllWindows()
        
        total_time = time.time() - start_time
        print(f"Обработка завершена!")
        print(f"Всего кадров: {frame_count}")
        print(f"Общее время: {total_time:.2f} сек")
        print(f"Средняя скорость: {frame_count/total_time:.2f} FPS")
    
    def draw_detections(self, frame, results):
        """
        Отрисовка bounding boxes и меток на кадре
        """
        for result in results:
            boxes = result.boxes
            if boxes is not None:
                for box in boxes:
                    # Координаты bounding box
                    x1, y1, x2, y2 = map(int, box.xyxy[0])
                    conf = box.conf[0].item()
                    class_id = int(box.cls[0])
                    
                    # Рисуем bounding box
                    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                    
                    # Добавляем метку с уверенностью
                    label = f"Person: {conf:.2f}"
                    label_size = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2)[0]
                    
                    # Фон для текста
                    cv2.rectangle(frame, (x1, y1 - label_size[1] - 10), (x1 + label_size[0], y1), (0, 255, 0), -1)
                    
                    # Текст
                    cv2.putText(frame, label, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 0), 2)
        
        return frame
    
    def display_info(self, frame, frame_count, fps):
        """
        Отображение информации на кадре
        """
        info_text = f"Frame: {frame_count} | FPS: {fps:.1f}"
        cv2.putText(frame, info_text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
        return frame

def main():
    parser = argparse.ArgumentParser(description='YOLOv8 Person Detection on Video')
    parser.add_argument('--video', type=str, required=True, help='Path to input video')
    parser.add_argument('--output', type=str, default='output.mp4', help='Path to output video')
    parser.add_argument('--model', type=str, default='yolov8m.pt', help='Path to YOLOv8 model')
    parser.add_argument('--conf', type=float, default=0.5, help='Confidence threshold')
    parser.add_argument('--no-show', action='store_true', help='Do not show video during processing')
    
    args = parser.parse_args()
    
    # Проверка существования видеофайла
    if not os.path.exists(args.video):
        print(f"Ошибка: Видеофайл {args.video} не найден!")
        return
    
    # Создание детектора
    detector = YOLOv8PersonDetector(model_path=args.model, conf_threshold=args.conf)
    
    # Обработка видео
    detector.process_video(
        video_path=args.video,
        output_path=args.output,
        show_video=not args.no_show
    )
# v = sort.save_interval_result(current_interval_start, interval_violations)
# print(v)
# Простой вариант использования без аргументов командной строки
def simple_example():
    """
    Простой пример использования без аргументов командной строки
    """
    video_path = "C:/Users/alex_dextop/PycharmProjects/Yolo_2/remont_30.mp4"  # Укажите путь к вашему видео
    output_path = "remont_1.mp4"
    
    # Проверка существования файла
    if not os.path.exists(video_path):
        print(f"Видеофайл {video_path} не найден!")
        print("Пожалуйста, укажите правильный путь к видеофайлу")
        return
    
    # Создание детектора
    detector = YOLOv8PersonDetector(model_path='yolov8m.pt', conf_threshold=0.5)
    
    # Обработка видео
    detector.process_video(
        video_path=video_path,
        output_path=output_path,
        show_video=True
    )
if __name__ == "__main__":
    # Использование с аргументами командной строки
    # main()
    
    # Или простой вариант (раскомментируйте нужный)
    simple_example()

Загрузка модели YOLOv8...
Модель загружена успешно!
Используется устройство: cpu
Параметры видео: 1280x720, FPS: 29.97, Кадров: 886
Начинаем обработку видео...

0: 384x640 3 persons, 207.9ms
Speed: 1.7ms preprocess, 207.9ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 persons, 224.1ms
Speed: 1.6ms preprocess, 224.1ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 persons, 199.8ms
Speed: 1.2ms preprocess, 199.8ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 persons, 191.5ms
Speed: 1.5ms preprocess, 191.5ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 persons, 187.7ms
Speed: 1.3ms preprocess, 187.7ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 persons, 193.7ms
Speed: 1.4ms preprocess, 193.7ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 persons, 137.5ms
Speed: 1.2ms preprocess, 137.5ms in

Трекинг лишь на класс птица: